### Regularising Measurements  

Hi Folks, 

### Problem

I've been trying to solve a problem where I have _measurements_ that I want to regularise. 

I have some prior knowledge that 

1. The mean of the measurements is should be around $\mu_0$ 
2. The measurements should be "close" to each other i.e. have low variance. 

### Potential Solution
The solution I have is to model the data as being iid with a Gaussian likelihood and parameterised by a mean $\mu$ and precision, $\tau = 1/\sigma^2$

$$ 
\begin{align}
x_i | \mu, \tau {} & \overset{iid}{\sim}  \mathcal{N}(\mu, \tau)  \\
\mu {} & \sim \mathcal{N}(\mu_0, n_0\tau) \\
\tau {} & \sim \textsf{Ga}(\alpha, \beta)
\end{align}
$$

Starting with parameters for $\mu_0$, $n_0$, $\alpha$, $\beta$, the posterior mean and precision can be calculated as 
$$
\begin{align}
\mu | \tau, x \sim {} & \mathcal{N}\left( \frac{n}{n+n_0}\bar{x} +  \frac{n_0}{n+n_0}\mu_0 , (n +n_0)\tau   \right) \\
\tau | x \sim  {} & \textsf{Ga}\left( \alpha + \frac{n}{2}, \beta + \frac{1}{2}\sum_i (x_i - \bar{x})^2 + \frac{nn_0}{2(n + n_0)}  (\bar{x} - \mu_0)^2  \right) 
\end{align}
$$
where $\bar{x}$ is the sample mean of the data. [1]

By setting $\mu_0$ to be equal to the prior knowledge of the mean and $n_0$, $\alpha$, $\beta$ arbirtrarily so that the variance of posterior is low, I, can compute the the posterior mean and precision. 


My solution is then to _adjust_ the data such that it matches the posterior mean and precision. 


1. Step 1: Find the $z$ scores with the mean and precision calculated from the data: 
    * $z^{\text{old}}_i  = \sqrt{\tau_x}(x - \bar{x})$

2. Step 2: Convert the $z$ scores back but with the posterior mean and precision 
    * $x^{\text{new}}_i  = \frac{1}{\sqrt{\tau}}z^{\text{old}}_i + \mu$

Is this valid? 

Is there a better way to do this?


### References 

[1] [The Conjugate Prior for the Normal Distribution](https://people.eecs.berkeley.edu/~jordan/courses/260-spring10/lectures/lecture5.pdf), Stat260 UC Berkeley


In [42]:
import pandas as pd
import numpy as np
import pymc3 as pm

In [37]:
mu0 = 3

n0 = 1
alpha = 10
beta = 1

tau_mean_prior = alpha/beta

print(f"Prior mean, mu0 {mu0}", f"Prior precision, tau0 {tau_mean_prior: .2f}")

x = np.array([1, 2, 4, 6])

n = len(x)
x_bar = np.mean(x)
tau_x = 1/np.var(x)

print("")

print(f"Data mean, x_bar {x_bar}", f"Data precision, tau_x {tau_x: .2f}")


alpha_post = alpha + n/2
beta_post = beta + 0.5*np.sum( (x - x_bar)**2) + 0.5*n*n0/(n + n0)*(x_bar - mu0)**2

tau_mean_post = alpha_post/beta_post
mu_mean_post = (n/(n + n0))*x_bar + (n0/(n + n0))*mu0

print("---")
print(f"Posterior mean, {mu_mean_post: .2f}", f"Posterior precision, {tau_mean_post: .2f}")


Prior mean, mu0 3 Prior precision, tau0  10.00

Data mean, x_bar 3.25 Data precision, tau_x  0.27
---
Posterior mean,  3.20 Posterior precision,  1.43


In [23]:
x = np.array([1, 2, 4, 6])

with pm.Model() as mod:
    # Priors
    mu0 = 3
    n0 = 1
    alpha = 10
    beta = 1
         
    tau = pm.Gamma("tau", alpha=alpha, beta=beta)
    mu = pm.Normal("mu", mu=mu0, tau=n0*tau)
    
    # Likelihood 
    y = pm.Normal("y", mu=mu, tau=tau, observed=x)
    
    # Magic     
    trace = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu, tau]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 19 seconds.


In [24]:
mu_sample_post = np.mean(trace.get_values("mu"))
tau_sample_post = np.mean(trace.get_values("tau"))
print("NUTS sampler results")
print(f"Posterior mean, {mu_sample_post: .2f}", f"Posterior precision, {tau_sample_post: .2f}")

NUTS sampler results
Posterior mean,  4.53 Posterior precision,  0.15


In [38]:
z_scores = np.sqrt(tau_x)*(x - x_bar)
z_scores

x_transformed = (1/np.sqrt(tau_mean_post))*z_scores + mu_mean_post
x_transformed

array([2.21968528, 2.65538071, 3.52677157, 4.39816243])